                                                 Домашнє завдання N 7

Так як ми працюємо з рекомендованними системами, по завданню нам радять використовувати бібліотеку Surprise, котра якраз є по суті додатком до знайомої нам бібліотеки scikit-learn для тренування моделей рекомендаційних систем.

Тому давайте імпортуємо цю бібліотеку і також завантажимо потрібний датасет movielens на 100 к даних. Думаю, нам буде достатньо цих даних для виконання домашнього завдання:

In [14]:
from surprise import Dataset, Reader, SVD, SVDpp, NMF, accuracy
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split

In [15]:
dataset_path = './ml-100k/u.data'
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file(dataset_path, reader=reader)

data

Давайте спробуємо вивести перші 5 рядків, щоб побачити, які дані містить наш датасет. Для цього використаємо трохи інший ніж зазвичай метод виводу:

In [16]:
N = 5
data_raw = data.raw_ratings

for i in range(N):
    print(data_raw[i])



('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


Окей, супер!

Тепер давайте побудуємо модел за допомогою 3 алгоритмів розграхунків: SVD, SVD++, NMF.

Почнемо з SVD:

In [17]:
algo = SVD()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9436  0.9458  0.9264  0.9307  0.9321  0.9357  0.0076  
MAE (testset)     0.7439  0.7452  0.7310  0.7323  0.7327  0.7370  0.0062  
Fit time          0.40    0.42    0.40    0.41    0.39    0.40    0.01    
Test time         0.04    0.08    0.04    0.07    0.04    0.05    0.02    


{'test_rmse': array([0.94355891, 0.94580925, 0.92637042, 0.9307119 , 0.93206893]),
 'test_mae': array([0.74391797, 0.74518466, 0.73096687, 0.73230874, 0.73274906]),
 'fit_time': (0.3966357707977295,
  0.41954708099365234,
  0.40049099922180176,
  0.4140768051147461,
  0.3933539390563965),
 'test_time': (0.04121708869934082,
  0.07804298400878906,
  0.04050016403198242,
  0.06846475601196289,
  0.03858208656311035)}

Тепер те саме зробимо з SVD++:

In [19]:
algo_svdpp = SVDpp()

cross_validate(algo_svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9129  0.9245  0.9243  0.9185  0.9173  0.9195  0.0044  
MAE (testset)     0.7172  0.7251  0.7274  0.7207  0.7189  0.7219  0.0038  
Fit time          6.22    6.20    6.17    6.20    6.20    6.20    0.02    
Test time         1.27    1.39    1.27    1.27    1.24    1.29    0.05    


{'test_rmse': array([0.91290831, 0.9245108 , 0.92433798, 0.91847836, 0.91729666]),
 'test_mae': array([0.71717114, 0.72509072, 0.72744894, 0.72069261, 0.71889752]),
 'fit_time': (6.219732999801636,
  6.20491886138916,
  6.173144102096558,
  6.197020769119263,
  6.202402114868164),
 'test_time': (1.272925853729248,
  1.3863019943237305,
  1.2734310626983643,
  1.2732832431793213,
  1.2384650707244873)}

І також використовуючи NMF модель:

In [18]:
algo_nmf = NMF()

cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9598  0.9682  0.9562  0.9636  0.9705  0.9637  0.0053  
MAE (testset)     0.7541  0.7606  0.7538  0.7575  0.7602  0.7572  0.0029  
Fit time          0.49    0.53    0.56    0.52    0.50    0.52    0.02    
Test time         0.03    0.07    0.03    0.06    0.07    0.05    0.02    


{'test_rmse': array([0.9598035 , 0.96819726, 0.95619876, 0.96356991, 0.97052899]),
 'test_mae': array([0.75412418, 0.76059835, 0.75377835, 0.75751824, 0.76021553]),
 'fit_time': (0.4933629035949707,
  0.5319468975067139,
  0.5617573261260986,
  0.5234808921813965,
  0.500504732131958),
 'test_time': (0.03200697898864746,
  0.0690619945526123,
  0.03126978874206543,
  0.06434202194213867,
  0.06702733039855957)}

Ми отримали результати оцінок RMSE та MAE всіх 3 алгоритмів. Давайте проаналізуємо отримані дані та зробимо висновки:

Алгоритм SVD: Має середнє значення RMSE приблизно 0.9345 і MAE приблизно 0.7373. Цей алгоритм показав непогані результати з помірним часом навчання та тестування.

Алгоритм SVD++: Показав кращі результати за RMSE (середнє 0.9194) та MAE (середнє 0.7213) порівняно з SVD, що свідчить про вищу точність прогнозування. Однак, час навчання та тестування значно вищий, що є важливим аспектом для великих наборів даних або обмежених обчислювальних ресурсів.

Алгоритм NMF: Має найвищі значення помилок RMSE (середнє 0.9648) та MAE (середнє 0.7572) серед трьох розглянутих алгоритмів, що вказує на нижчу точність прогнозування порівняно з SVD та SVD++. Час навчання та тестування подібний до SVD.

Як висновок, можемо зазначити, що найбільш вдалим тут здається вибір алгоритму SVD++, так як в нас датасет не такий великий. Якби ми взяли датасет не на 100к а на 1М даних, то досить велика ймовірність, що краще б було тоді застосовувати алгоритм SVD.

Ми знайшли оптимальний алгоритм на основі пошуку параметрів RMSE,MAE через cross_validation.

Тепер же давайте застосуємо метод GridSearchCV для пошуку оптимальних гіперпараметрів. Для цього використаємо розділ документації бібліотеки surprise: Tune algorithm parameters with GridSearchCV.

Зробимо це для алгоритму SVD++, так як він показався найбільш оптимальним, хоча достатньо ресурсно-залежний:

In [25]:
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(f"RMSE for SVD++: {gs.best_score['rmse']}")
print(gs.best_params['rmse'])
print(f"MAE for SVD++: {gs.best_score['mae']}")
print(gs.best_params['mae'])

algo_rmse_svd_pp = SVDpp(**gs.best_params['rmse'])
algo_mae_svd_pp = SVDpp(**gs.best_params['mae'])

RMSE for SVD++: 0.9639470856624867
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
MAE for SVD++: 0.7725249391418725
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


Тепер, коли у нас є модель, яка оптимізована за обома метриками і підібрані гіперпараметри, ми можемо провести оцінку на тестовому наборі для перевірки точності. Для цього розділимо наш датасет на тренувальний і тестовий у пропорції 80 на 20:

In [26]:
trainset, testset = train_test_split(data, test_size=0.2)

Тепер навчимо нашу модель на тренувальному наборі:

In [27]:
algo_rmse_svd_pp.fit(trainset)
algo_mae_svd_pp.fit(trainset)

Тепер отримаємо прогнози на нашому тестовому наборі:

In [28]:
predictions_rmse = algo_rmse_svd_pp.test(testset)
predictions_mae = algo_mae_svd_pp.test(testset)

І тепер проведемо оцінку моделі:

In [29]:
accuracy_rmse_svd_pp = accuracy.rmse(predictions_rmse)
accuracy_mae_svd_pp = accuracy.mae(predictions_mae)

RMSE: 0.9660
MAE:  0.7739


Давайте зробимо пошук оптимальних гіперпараметрів для алгоритмів SVD та NMF і потім оцінимо отримані результати.

Для алгоритму SVD:

In [31]:
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(f"RMSE for SVD: {gs.best_score['rmse']}")
print(gs.best_params['rmse'])
print(f"MAE for SVD: {gs.best_score['mae']}")
print(gs.best_params['mae'])

algo_rmse_svd = SVD(**gs.best_params['rmse'])
algo_mae_svd = SVD(**gs.best_params['mae'])

RMSE for SVD: 0.9639574246997262
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
MAE for SVD: 0.772689612058347
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [32]:
trainset, testset = train_test_split(data, test_size=0.2)

algo_rmse_svd.fit(trainset)
algo_mae_svd.fit(trainset)

predictions_rmse = algo_rmse_svd.test(testset)
predictions_mae = algo_mae_svd.test(testset)

accuracy_rmse_svd = accuracy.rmse(predictions_rmse)
accuracy_mae_svd = accuracy.mae(predictions_mae)

RMSE: 0.9573
MAE:  0.7693


Для алгоритму NMF:

In [34]:
param_grid = {"n_epochs": [5, 10], "n_factors": [15, 50, 100], "reg_pu": [0.05, 0.1, 0.15], "reg_qi": [0.05, 0.1, 0.15]}
gs = GridSearchCV(NMF, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(f"RMSE for NMF: {gs.best_score['rmse']}")
print(gs.best_params['rmse'])
print(f"MAE for NMF: {gs.best_score['mae']}")
print(gs.best_params['mae'])

algo_rmse_nmf = NMF(**gs.best_params['rmse'])
algo_mae_nmf = NMF(**gs.best_params['mae'])

RMSE for NMF: 0.9688716985963349
{'n_epochs': 10, 'n_factors': 15, 'reg_pu': 0.15, 'reg_qi': 0.15}
MAE for NMF: 0.7622409007043022
{'n_epochs': 10, 'n_factors': 15, 'reg_pu': 0.1, 'reg_qi': 0.15}


In [35]:
trainset, testset = train_test_split(data, test_size=0.2)

algo_rmse_nmf.fit(trainset)
algo_mae_nmf.fit(trainset)

predictions_rmse = algo_rmse_nmf.test(testset)
predictions_mae = algo_mae_nmf.test(testset)

accuracy_rmse_nmf = accuracy.rmse(predictions_rmse)
accuracy_mae_nmf = accuracy.mae(predictions_mae)

RMSE: 0.9618
MAE:  0.7550


Тепер виведемо усі RMSE,MAE для всіх 3-х алгоритмів:

In [38]:
print("SVD++:")
accuracy_rmse_svd_pp = accuracy.rmse(predictions_rmse)
accuracy_mae_svd_pp = accuracy.mae(predictions_mae)
print("\nSVD:")
accuracy_rmse_svd = accuracy.rmse(predictions_rmse)
accuracy_mae_svd = accuracy.mae(predictions_mae)
print("\nNMF:")
accuracy_rmse_nmf = accuracy.rmse(predictions_rmse)
accuracy_mae_nmf = accuracy.mae(predictions_mae)

SVD++:
RMSE: 0.9618
MAE:  0.7550

SVD:
RMSE: 0.9618
MAE:  0.7550

NMF:
RMSE: 0.9618
MAE:  0.7550


Здається, алгоритм SVD показав найкращий результат за метрикою RMSE (0.9573) та майже найкращий за MAE (0.7693), що свідчить про добру здатність моделі загалом передбачати оцінки, особливо з точки зору квадратичних помилок.

По методу cross_validation найкращим був SVD++. 
Можливо, це повʼязано з тим, що в param_grid для GridSearchCV я задав не найкращі параметри для SVD++...